# Runge-Kutta
The `rungekutta` module provides implementations of `Runge-Kutta` time integrators that  transform a symbolic expression into a ready to use RK "rates" and "update" formulas.

To demonstrate how to use this module, we consider the following test problem and solve it using each of the available time integrators

$
\frac{d\mathbf{u}}{dt} = -s\mathbf{u}
$

To get started, we import the `rungekutta` module

In [ ]:
from openfd import rungekutta


# 2N Low-storage Runge-Kutta schemes
A 2N low-storage Runge-Kutta scheme requires twice as much as memory as the solution itself. In this case, to set up the problem, in addition to defining `u` we also need to define an additional variable `du`. This additional variable represents temporary memory that will be used by the low storage scheme to update the solution.

To advance the solution in time from `t` to `t  + dt`, the scheme proceeds in two steps: 
 * The first step sets the rates (evaluates the right-hand side)
 * The second step updates the solution
 
The `rate` and `solution` update are repeated for the number of stages, which we will return to when running hte code.

In [ ]:
from sympy import symbols
from openfd import GridFunction
n = symbols('n')
u = GridFunction('u', shape=(n,))
s = GridFunction('s', shape=(n,))
du = GridFunction('du', shape=(n,))

Recall that the problem we are trying to solve involves the following right-hand side

In [ ]:
rhs = -s*u

Once the problem setup is complete, we initialize a new RK object that will be used in the proceeding sections

In [ ]:
rk = rungekutta.LSRK4()

## Setting rates and computing update
We are now ready to generate the expression that sets the RK rates

In [ ]:

du, rate_expr = rk.rates(du, rhs)
rate_expr 


If the right-hand side needs to be broken up into multiple evaluations, then use `append=True` for the proceeding evaluations.


In [ ]:
du, rate_expr2 = rk.rates(du, -u, append=True)
rate_expr2

We also need to update the solution at the end of each stage, and the expression for that is simply obtained by

In [ ]:
u, upd_expr = rk.update(u, du)
print(upd_expr)

In the next section, we will streamline the process a little bit more (not defining any intermediate variables `rate_expr` etc).

## Kernel generation 
We will construct and execute `Cuda` kernels, but changing to any other generator can easily be accomplished by changing `CudaGenerator` to for example `OpenclGenerator`.

In [ ]:
from openfd.dev import kernelgenerator as kg
from openfd import Bounds
generator = kg.CudaGenerator
kernels = []

In [ ]:
# lhs, rhs, grid dimension symbols, bounds, and rates
krate = generator((n,), Bounds(n), *rk.rates(du, rhs))
# .. update
kupd = generator((n,), Bounds(n), *rk.update(u, du))

Once we have prepared the kernels we need to select what regions to generate. Since the computation is homogeneous, we set the region to `1`.

In [ ]:
kernels.append(krate.kernel('rates', 1))
kernels.append(kupd.kernel('update', 1))

We can inspect the kernels by calling their `code` attribute

In [ ]:
print(kernels[0].code)

In [ ]:
print(kernels[1].code)

## Kernel evaluation
Once the kernels have been generated, it is time to execute them. To start off, we initialize a compatible evaluator (same language as the one used to generate the kernels). 



In [ ]:
from openfd.dev import cudaevaluator as ce
evaluator = ce.CudaEvaluator

Next, we allocate memory for all gridfunctions and assign some initial values to them.

In [ ]:
import numpy as np
nmem = np.int32(32)
gpu_u = np.array(nmem).astype(np.float32)
gpu_du = np.array(nmem).astype(np.float32)
gpu_s = np.ones((nmem,)).astype(np.float32)
dt = np.float32(0.1)


We pass the kernels to the evaluator and bind the all of the input and out arguments to their symbols and values. If you forget some input argument, then the `evaluator` will tell you which one you forgot to specify.

In [ ]:

ke = evaluator(kernels, inputs={n : nmem, s : gpu_s, u : gpu_u, du : gpu_du,
                                rk.ak : np.float32(rk.a[0]), 
                                rk.bk : np.float32(rk.b[0]), rk.dt : dt }, outputs={du : gpu_du})

Next, we evaluate and check the result

In [ ]:
ke.eval()

In [ ]:
#TODO: Continue this notebook once the evaluator is up and running properly